In [1]:
from dotenv import load_dotenv
load_dotenv('../.vscode/.env')

True

In [2]:
import pandas as pd

/var/folders/qb/cr87d2s56q51lk72620820hw0000gn/T/ipykernel_17240/4080736814.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [3]:
df = pd.read_csv('../data/scrapped_recipes/scrapped_recipes.csv')

In [21]:
row = df.iloc[175]

In [24]:
ASSISTANT_INSTRUCTION = f'''
You are helpful diabetic-friendly recipe assistant who give different recipe each time when user ask. Recipe can be for vegetarian, vegan, and non-vegeterian diets.You generate diabetic friendly recipe in below format:
<recipe>
    <recipe_name> {row['recipe_name']} </recipe_name>
    <ingredients> {row['ingredients']} </ingredients>
    <directions> {row['directions']} </directions>
    <nutrition> {row['nutrients']} </nutrition> 
</recipe>
Always use above format to give recipe. Give total calories and calorie per serving.
'''

In [27]:
ASSISTANT_NAME = 'Recipe Assistant'

In [28]:
print(ASSISTANT_INSTRUCTION)


You are helpful diabetic-friendly recipe assistant who give different recipe each time when user ask. Recipe can be for vegetarian, vegan, and non-vegeterian diets.You generate diabetic friendly recipe in below format:
<recipe>
    <recipe_name> Chicken cordon bleu </recipe_name>
    <ingredients> 4 chicken breasts (4 ounces each),1 cup panko bread crumbs,1 teaspoon chopped fresh parsley,1/2 teaspoon onion powder,1/2 teaspoon garlic powder,1/2 teaspoon paprika,1/2 teaspoon kosher salt,1/4 teaspoon ground black pepper,1/2 cup egg substitute,2 ounces sliced Swiss cheese,2 ounces sliced Canadian bacon,1 cup sauteed spinach </ingredients>
    <directions> Heat oven to 400 F. Lightly coat a baking sheet with cooking spray; set aside.,In a small bowl, combine the bread crumbs, parsley, onion powder, garlic powder, paprika, salt and pepper. In another small bowl, add the egg substitute.,Pound out each chicken breast to an even thickness with a meat mallet and lay flat on a cutting board. Pla

In [29]:
client = OpenAI(api_key=os.environ.get("OPENAI_API_KEY", ""))


assistant = client.beta.assistants.create(
    name=ASSISTANT_NAME,
    instructions=ASSISTANT_INSTRUCTION,
    model="gpt-4-1106-preview",
)
show_json(assistant)

{'id': 'asst_OuIPIcRa8eI0n378kObBpb5v',
 'created_at': 1712204498,
 'description': None,
 'file_ids': [],
 'instructions': '\nYou are helpful diabetic-friendly recipe assistant who give different recipe each time when user ask. Recipe can be for vegetarian, vegan, and non-vegeterian diets.You generate diabetic friendly recipe in below format:\n<recipe>\n    <recipe_name> Chicken cordon bleu </recipe_name>\n    <ingredients> 4 chicken breasts (4 ounces each),1 cup panko bread crumbs,1 teaspoon chopped fresh parsley,1/2 teaspoon onion powder,1/2 teaspoon garlic powder,1/2 teaspoon paprika,1/2 teaspoon kosher salt,1/4 teaspoon ground black pepper,1/2 cup egg substitute,2 ounces sliced Swiss cheese,2 ounces sliced Canadian bacon,1 cup sauteed spinach </ingredients>\n    <directions> Heat oven to 400 F. Lightly coat a baking sheet with cooking spray; set aside.,In a small bowl, combine the bread crumbs, parsley, onion powder, garlic powder, paprika, salt and pepper. In another small bowl, a

In [10]:
from openai import OpenAI
import os

In [11]:
import json

def show_json(obj):
    display(json.loads(obj.model_dump_json()))

In [30]:
ASSISTANT_ID='asst_OuIPIcRa8eI0n378kObBpb5v'

In [31]:
from bs4 import BeautifulSoup
client = OpenAI(api_key=os.environ.get("OPENAI_API_KEY", ""))

def submit_message(assistant_id, thread, user_message):
    client.beta.threads.messages.create(
        thread_id=thread.id, role="user", content=user_message
    )
    return client.beta.threads.runs.create(
        thread_id=thread.id,
        assistant_id=assistant_id,
    )


def get_response(thread):
    messages = client.beta.threads.messages.list(thread_id=thread.id)
    for m in messages:
        if m.role == 'assistant':
            return m.content[0].text.value
# Write recipe in xml format using Beautiful soup
def write_recipe(id: str, recipe_content:str):
    soup = BeautifulSoup(recipe_content, "xml")
    with open(f'../data/scrapped_recipes/{id}.xml', 'w') as f:
        f.write(soup.prettify())

def create_thread_and_run(assistant_id: str, user_input:str):
    thread = client.beta.threads.create()
    run = submit_message(assistant_id, thread, user_input)
    return thread, run

In [32]:
import time

def wait_on_run(run, thread):
    while run.status == "queued" or run.status == "in_progress":
        run = client.beta.threads.runs.retrieve(
            thread_id=thread.id,
            run_id=run.id,
        )
        time.sleep(0.5)
    return run

In [33]:
prompts = [
    "Can you share a diabetic-friendly recipe for a hearty morning meal that's both satisfying and nutritious?",
    "I'm looking for a diabetic-friendly lunch option that's easy to prepare and delicious. Any suggestions?",
    "What's a diabetic-friendly recipe for dinner that's full of flavor and incorporates plenty of vegetables and lean proteins?",
    "Do you have a diabetic-friendly recipe for a quick and energizing breakfast that doesn't spike blood sugar levels?",
    "I'm in need of a diabetic-friendly lunch recipe that's suitable for meal prep and can be enjoyed on-the-go. Any recommendations?",
    "Could you recommend a diabetic-friendly dinner recipe that's perfect for entertaining guests, yet won't compromise on taste or healthiness?",
    "I'm searching for a diabetic-friendly breakfast recipe that's warm and comforting, ideal for those chilly mornings. Any ideas?",
    "What's a diabetic-friendly recipe for a light yet filling lunch option that's perfect for busy weekdays?",
    "Can you suggest a diabetic-friendly recipe for dinner that's family-friendly and can be customized to suit different taste preferences?",
    "Do you have a diabetic-friendly breakfast recipe that's suitable for someone with a sweet tooth but won't cause a sugar spike?"
]

In [34]:
for i in range(11,21):
    thread, run = create_thread_and_run(assistant_id= ASSISTANT_ID, user_input=prompts[i-1-10])
    run = wait_on_run(thread=thread, run = run)
    recipe_content = get_response(thread=thread)
    write_recipe(id=str(i), recipe_content=recipe_content)
    recipe_id = str(i)
    print(f'Generated recipe for id:{recipe_id}')

Generated recipe for id:11
Generated recipe for id:12
Generated recipe for id:13
Generated recipe for id:14
Generated recipe for id:15
Generated recipe for id:16
Generated recipe for id:17
Generated recipe for id:18
Generated recipe for id:19
Generated recipe for id:20


In [35]:
import re

# Function to extract nutritional values
def extract_nutrition_values(text):
    # Dictionary to hold the extracted values
    nutrition_values = {}

    # Regular expressions for extracting fat, carbohydrate, and protein
    patterns = {
        'fat': r"Total Fat (\d+\.?\d*g)",
        'carbohydrate': r"Total Carbohydrate (\d+\.?\d*g)",
        'protein': r"Protein (\d+\.?\d*g)",
    }

    # Loop through the patterns and search in the text
    for key, pattern in patterns.items():
        match = re.search(pattern, text, re.IGNORECASE)
        if match:
            # If a match is found, add it to the dictionary
            nutrition_values[key] = match.group(1)
        else:
            # If no match is found, set the value to None
            nutrition_values[key] = None

    return nutrition_values

In [36]:
import glob
from bs4 import  BeautifulSoup
file_paths = glob.glob(pathname='../data/scrapped_recipes/*.xml')

In [37]:
file_paths

['../data/scrapped_recipes/8.xml',
 '../data/scrapped_recipes/9.xml',
 '../data/scrapped_recipes/14.xml',
 '../data/scrapped_recipes/15.xml',
 '../data/scrapped_recipes/17.xml',
 '../data/scrapped_recipes/16.xml',
 '../data/scrapped_recipes/12.xml',
 '../data/scrapped_recipes/13.xml',
 '../data/scrapped_recipes/11.xml',
 '../data/scrapped_recipes/10.xml',
 '../data/scrapped_recipes/20.xml',
 '../data/scrapped_recipes/18.xml',
 '../data/scrapped_recipes/19.xml',
 '../data/scrapped_recipes/4.xml',
 '../data/scrapped_recipes/5.xml',
 '../data/scrapped_recipes/7.xml',
 '../data/scrapped_recipes/6.xml',
 '../data/scrapped_recipes/2.xml',
 '../data/scrapped_recipes/3.xml',
 '../data/scrapped_recipes/1.xml']

In [38]:
from pydantic import BaseModel
from typing import  List, Optional
class Nutrition(BaseModel):
    fat: Optional[str] = None
    protein: Optional[str] = None
    carbohydrate: Optional[str] = None

class Recipe(BaseModel):
    id: str
    recipe_name:str
    ingredients: str
    directions: str
    nutrition: Nutrition 

class Recipes(BaseModel):
    recipes: List[Recipe] = []

In [39]:
def read_xml_file(file_path: str):
    with open(file_path, 'r') as f:
        file = f.read() 
        return file

def clean_text(content: str):
    clean_content = content.strip()
    return clean_content

In [40]:
from pathlib import Path
def parse_llm_recipes(file_paths:list):
    recipes = Recipes()
    for file_path in file_paths:
        print(f'Parsing file:{file_path}')
        id = Path(file_path).stem
        file = read_xml_file(file_path)
        soup = BeautifulSoup(file, 'xml')
        recipe_name = soup.find('recipe_name')
        recipe_name = clean_text(recipe_name.text)
        ingredients = soup.find('ingredients')
        ingredients = clean_text(ingredients.text)
        directions = soup.find('directions')
        directions = clean_text(directions.text)
        nutrition = soup.find('nutrition')
        nutrition = clean_text(nutrition.text)
        parsed_nutrition = extract_nutrition_values(text=nutrition)
        nutrition = Nutrition(fat=parsed_nutrition['fat'], protein=parsed_nutrition['protein'],carbohydrate=parsed_nutrition['carbohydrate'])
        recipe = Recipe(id = id, recipe_name = recipe_name, ingredients = ingredients,directions=directions, nutrition=nutrition)
        recipes.recipes.append(recipe) 
    return recipes 

In [41]:
llm_recipes = parse_llm_recipes(file_paths)

Parsing file:../data/scrapped_recipes/8.xml
Parsing file:../data/scrapped_recipes/9.xml
Parsing file:../data/scrapped_recipes/14.xml
Parsing file:../data/scrapped_recipes/15.xml
Parsing file:../data/scrapped_recipes/17.xml
Parsing file:../data/scrapped_recipes/16.xml
Parsing file:../data/scrapped_recipes/12.xml
Parsing file:../data/scrapped_recipes/13.xml
Parsing file:../data/scrapped_recipes/11.xml
Parsing file:../data/scrapped_recipes/10.xml
Parsing file:../data/scrapped_recipes/20.xml
Parsing file:../data/scrapped_recipes/18.xml
Parsing file:../data/scrapped_recipes/19.xml
Parsing file:../data/scrapped_recipes/4.xml
Parsing file:../data/scrapped_recipes/5.xml
Parsing file:../data/scrapped_recipes/7.xml
Parsing file:../data/scrapped_recipes/6.xml
Parsing file:../data/scrapped_recipes/2.xml
Parsing file:../data/scrapped_recipes/3.xml
Parsing file:../data/scrapped_recipes/1.xml


In [42]:
import json
with open('../data/scrapped_recipes/gpt4_1106_recipes_v2.json', 'w') as f:
    json.dump(llm_recipes.model_dump(mode='json'), f, indent=4, ensure_ascii=False)

In [43]:
import json
with open('../data/scrapped_recipes/gpt4_1106_recipes_v2.json') as f:
    gpt4_recipe = json.load(f)

In [44]:
df_gpt4_recipe = pd.DataFrame.from_records(gpt4_recipe['recipes'])


In [45]:
df_gpt4_recipe

,id,recipe_name,ingredients,directions,nutrition
0,8,Chicken and Avocado Salad,"2 cups cooked chicken breast, diced, 1 ripe av...","In a large bowl, combine the diced chicken, av...","{'fat': '18g', 'protein': '22g', 'carbohydrate..."
1,9,Stuffed Bell Peppers,"4 large bell peppers (any color), 1 cup quinoa...",Preheat oven to 375°F (190°C). Cut the tops of...,"{'fat': '8g', 'protein': '14g', 'carbohydrate'..."
2,14,Scrambled Tofu with Spinach and Tomato,"1 block (14 ounces) firm tofu, drained and cru...",Heat a nonstick skillet over medium heat and a...,"{'fat': '12g', 'protein': '19g', 'carbohydrate..."
3,15,Quinoa Chickpea Salad Jars,"1 cup quinoa, rinsed,2 cups water,1 can (15 oz...","In a medium saucepan, combine quinoa and water...","{'fat': '10g', 'protein': '15g', 'carbohydrate..."
4,17,Spinach and Mushroom Frittata,"6 egg whites,2 whole eggs,1 cup fresh spinach,...","Preheat your oven to 350 F.,In a mixing bowl, ...","{'fat': '24g', 'protein': '54g', 'carbohydrate..."
5,16,Stuffed Bell Peppers with Quinoa and Black Beans,"4 large bell peppers, tops cut away and seeds ...",Preheat oven to 375 F. Place the bell peppers ...,"{'fat': '9g', 'protein': '16g', 'carbohydrate'..."
6,12,Mediterranean Chickpea Salad,"2 cups chickpeas, drained and rinsed,1 cup che...","In a large bowl, combine chickpeas, cherry tom...","{'fat': '14g', 'protein': '11g', 'carbohydrate..."
7,13,Grilled Lemon Herb Chicken Salad,"2 boneless, skinless chicken breasts (6 ounces...",Preheat grill or grill pan over medium-high he...,"{'fat': '25g', 'protein': '50g', 'carbohydrate..."
8,11,Spinach and Mushroom Frittata,"6 large eggs, 1/4 cup non-fat milk, 1 tablespo...","Preheat the oven to 375 F. In a bowl, whisk to...","{'fat': '60g', 'protein': '78g', 'carbohydrate..."
9,10,Almond Flour Pancakes,"1 cup almond flour, 2 large eggs, 1/4 cup wate...","In a medium mixing bowl, whisk together the al...","{'fat': '16g', 'protein': '7g', 'carbohydrate'..."


In [46]:
recipe = pd.read_csv('../data/scrapped_recipes/scrapped_recipes.csv')


In [47]:
recipe['id'] = recipe.index + 1

In [48]:
recipe = recipe[['id','recipe_name', 'source_url','ingredients','directions','serving_size']]

In [49]:
recipe

,id,recipe_name,source_url,ingredients,directions,serving_size
0,1,Artichoke dip,https://www.mayoclinic.org/healthy-lifestyle/r...,"1 can (15.5 ounces) artichoke hearts in water,...","In a mixing bowl, combine the ingredients. Tra...",Serving size: About 1/2 cup
1,2,"Artichoke, spinach and white bean dip",https://www.mayoclinic.org/healthy-lifestyle/r...,"2 cups artichoke hearts,1 tablespoon black pep...","Heat oven to 350 degrees.,Mix all ingredients ...",Serving size: About 1/2 cup
2,3,Artichokes alla Romana,https://www.mayoclinic.org/healthy-lifestyle/r...,"2 cups fresh breadcrumbs, preferably whole-whe...","Heat the oven to 400 F. In a bowl, combine the...",Serving size: 1/4 artichoke
3,4,Avocado dip,https://www.mayoclinic.org/healthy-lifestyle/r...,"1/2 cup fat-free sour cream,2 teaspoons choppe...","In a small bowl, combine sour cream, onion, ho...",Serving size: 1/4 cup
4,5,Baba ghanoush,https://www.mayoclinic.org/healthy-lifestyle/r...,"1 bulb garlic (about 8 cloves),2 eggplants, sl...",Spray cold grill with cooking spray. Heat one ...,Serving size: Half a pita and 2 tablespoons dip
...,...,...,...,...,...,...
520,521,"Vegetable, lentil and garbanzo bean stew",https://www.mayoclinic.org/healthy-lifestyle/r...,3 cups butternut squash (approximately 1 1/2 -...,"In a Dutch oven, slowly sweat vegetables (squa...",Serving size: About 2 cups
521,522,Vegetarian chili,https://www.mayoclinic.org/healthy-lifestyle/r...,"2 cups diced onion,1 cup diced celery,1 cup di...","In a stockpot over low heat, cook onion, celer...",Serving size: About 2 cups
522,523,White chicken chili,https://www.mayoclinic.org/healthy-lifestyle/r...,"1 can (10 ounces) white chunk chicken,2 cans (...","In a large soup pot, add the chicken, beans, t...",Serving size: About 1 1/2 cups
523,524,Wild rice mushroom soup,https://www.mayoclinic.org/healthy-lifestyle/r...,"1 tablespoon olive oil,Half a white onion, cho...",Put olive oil in stockpot and bring to medium ...,Serving size: About 1 1/2 cups


In [50]:
dataset_recipes = recipe[['id','recipe_name']]


In [51]:
dataset_recipes['recipe_name'] = dataset_recipes['recipe_name'].map(lambda x : x.lower())


/var/folders/qb/cr87d2s56q51lk72620820hw0000gn/T/ipykernel_17240/3773327544.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset_recipes['recipe_name'] = dataset_recipes['recipe_name'].map(lambda x : x.lower())


In [52]:
dataset_recipes

,id,recipe_name
0,1,artichoke dip
1,2,"artichoke, spinach and white bean dip"
2,3,artichokes alla romana
3,4,avocado dip
4,5,baba ghanoush
...,...,...
520,521,"vegetable, lentil and garbanzo bean stew"
521,522,vegetarian chili
522,523,white chicken chili
523,524,wild rice mushroom soup


In [53]:
gpt4_recipes_sub = df_gpt4_recipe[['id','recipe_name']]
gpt4_recipes_sub['recipe_name'] = gpt4_recipes_sub['recipe_name'].map(lambda x : x.lower())


/var/folders/qb/cr87d2s56q51lk72620820hw0000gn/T/ipykernel_17240/54345361.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gpt4_recipes_sub['recipe_name'] = gpt4_recipes_sub['recipe_name'].map(lambda x : x.lower())


In [58]:
# Get frittata recipe
chicken_recipes = []
for index, row in gpt4_recipes_sub.iterrows():
    if 'chicken salad' in row['recipe_name']:
        chicken_recipes.append({'id':row['id'],'recipe_name': row['recipe_name']})

In [59]:
chicken_recipes

[{'id': '13', 'recipe_name': 'grilled lemon herb chicken salad'},
 {'id': '2', 'recipe_name': 'grilled chicken salad with avocado dressing'}]

In [60]:
dataset_chicken_recipes = []
for index, row in dataset_recipes.iterrows():
    if 'chicken salad' in row['recipe_name']:
        dataset_chicken_recipes.append({'id':row['id'],'recipe_name': row['recipe_name']})

In [61]:
dataset_chicken_recipes

[{'id': 180,
  'recipe_name': 'chicken salad with pineapple and balsamic vinaigrette'},
 {'id': 181, 'recipe_name': 'chicken salad with thai flavors'},
 {'id': 208, 'recipe_name': 'grilled chicken salad with olives and oranges'},
 {'id': 348,
  'recipe_name': 'chicken salad with pineapple and balsamic vinaigrette'},
 {'id': 349, 'recipe_name': 'chicken salad with thai flavors'},
 {'id': 360, 'recipe_name': 'grilled chicken salad with buttermilk dressing'},
 {'id': 361, 'recipe_name': 'grilled chicken salad with olives and oranges'},
 {'id': 387, 'recipe_name': 'buffalo chicken salad wrap'}]

In [62]:
import json
import logging
import os

import openai
import wikipedia

from typing import Optional
from IPython.display import display, Markdown
from tenacity import retry, wait_random_exponential, stop_after_attempt

logging.basicConfig(level=logging.INFO, format=' %(asctime)s - %(levelname)s - %(message)s')

OPENAI_MODEL = 'gpt-3.5-turbo-0613'

In [63]:
labels = [
    "person",      # people, including fictional characters
    "fac",         # buildings, airports, highways, bridges
    "org",         # organizations, companies, agencies, institutions
    "gpe",         # geopolitical entities like countries, cities, states
    "loc",         # non-gpe locations
    "product",     # vehicles, foods, appareal, appliances, software, toys 
    "event",       # named sports, scientific milestones, historical events
    "work_of_art", # titles of books, songs, movies
    "law",         # named laws, acts, or legislations
    "language",    # any named language
    "date",        # absolute or relative dates or periods
    "time",        # time units smaller than a day
    "percent",     # percentage (e.g., "twenty percent", "18%")
    "money",       # monetary values, including unit
    "quantity",    # measurements, e.g., weight or distance
]

In [64]:
def system_message(labels):
    return f"""
You are an expert in Natural Language Processing. Your task is to identify common Named Entities (NER) in a given text.
The possible common Named Entities (NER) types are exclusively: ({", ".join(labels)})."""


In [65]:
def assisstant_message():
    return f"""
EXAMPLE:
    Text: '1 pie crust (store-bought or homemade), 6 cups thinly sliced apples (such as Granny Smith or a combination of tart and sweet apples), 3/4 cup granulated sugar, 2 tbsp all-purpose flour, 1 tsp ground cinnamon, 1/4 tsp ground nutmeg, 1/4 tsp salt, 1 tsp vanilla extract, 1/2 cup unsalted butter (cold and cut into small pieces), 3/4 cup all-purpose flour (for crumb topping), 1/2 cup brown sugar (packed, for crumb topping), 1/4 tsp baking powder (for crumb topping), 1/4 tsp salt (for crumb topping), Optional: Vanilla ice cream or whipped cream for serving'
    {{
        "product": ["pie crust","apples", "granulated sugar", "all-purpose flour", "ground cinnamon", "ground nutmeg", "salt", "vanilla extract", "unsalted water","brown sugar","baking powder", "vanilla ice cream", "whipped cream"],
    }}
--"""

In [66]:
def user_message(text):
    return f"""
TASK:
    Text: {text}
"""

In [67]:
def recipe_ingredients(text: str, label_entities: dict) -> list:
    """
    Recipe ingredients
    """
    return label_entities['product']

In [68]:
def generate_functions(labels: dict) -> list:
    return [
        {   
            "type": "function",
            "function": {
                "name": "recipe_ingredients",
                "description": "Enrich Text with recipe ingredients",
                "parameters": {
                    "type": "object",
                        "properties": {
                            "r'^(?:' + '|'.join({labels}) + ')$'": 
                            {
                                "type": "array",
                                "items": {
                                    "type": "string"
                                }
                            }
                        },
                        "additionalProperties": False
                },
            }
        }
    ]

In [69]:
@retry(wait=wait_random_exponential(min=1, max=10), stop=stop_after_attempt(5))
def run_openai_task(labels, text):
    messages = [
          {"role": "system", "content": system_message(labels=labels)},
          {"role": "assistant", "content": assisstant_message()},
          {"role": "user", "content": user_message(text=text)}
      ]

    # TODO: functions and function_call are deprecated, need to be updated
    # See: https://platform.openai.com/docs/api-reference/chat/create#chat-create-tools
    response = openai.chat.completions.create(
        model="gpt-3.5-turbo-0613",
        messages=messages,
        tools=generate_functions(labels),
        tool_choice={"type": "function", "function" : {"name": "recipe_ingredients"}}, 
        temperature=0,
        frequency_penalty=0,
        presence_penalty=0,
    )

    response_message = response.choices[0].message
    
    available_functions = {"recipe_ingredients": recipe_ingredients}  
    function_name = response_message.tool_calls[0].function.name
    
    function_to_call = available_functions[function_name]
    logging.info(f"function_to_call: {function_to_call}")

    function_args = json.loads(response_message.tool_calls[0].function.arguments)
    logging.info(f"function_args: {function_args}")

    function_response = function_to_call(text, function_args)

    return {"model_response": response, 
            "function_response": function_response}

In [70]:
text = """6 cups thinly sliced apples, 1 tablespoon lemon juice (Optional), ¾ cup white sugar, 2 tablespoons all-purpose flour, ½ teaspoon ground cinnamon, ⅛ teaspoon ground nutmeg, ½ cup raisins (Optional), ½ cup chopped walnuts (Optional), 1 (9 inch) pie shell, ½ cup all-purpose flour, ½ cup packed brown sugar, 3 tablespoons butter"""
result = run_openai_task(labels, text)

 2024-04-03 21:30:16,261 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
 2024-04-03 21:30:16,264 - INFO - function_to_call: <function recipe_ingredients at 0x11fbf0e00>
 2024-04-03 21:30:16,264 - INFO - function_args: {'product': ['apples', 'lemon juice', 'white sugar', 'all-purpose flour', 'ground cinnamon', 'ground nutmeg', 'raisins', 'chopped walnuts', 'pie shell', 'packed brown sugar', 'butter']}


In [71]:
result['function_response']

['apples',
 'lemon juice',
 'white sugar',
 'all-purpose flour',
 'ground cinnamon',
 'ground nutmeg',
 'raisins',
 'chopped walnuts',
 'pie shell',
 'packed brown sugar',
 'butter']

In [72]:
def word_match_count(arr1, arr2):
    # Split each phrase in both arrays into words and flatten the list of lists
    words1 = set(word for phrase in arr1 for word in phrase.split())
    words2 = set(word for phrase in arr2 for word in phrase.split())
    
    # Find the intersection of the two sets to get the matching words
    matching_words = words1.intersection(words2)
    
    # Return the matching words and their count
    return matching_words, len(matching_words)

In [94]:
recipe_authenticity_result = []
for item1 in chicken_recipes:
    item_recipe = {'id': item1['id'], 'recipe_name':item1['recipe_name'], 'matched_recipes':[]}
    ingredients1 = str(df_gpt4_recipe[df_gpt4_recipe['id'] == str(item1['id'])]['ingredients'].values[0])
    result1 = run_openai_task(labels, ingredients1)
    ingredients1_arr = result1['function_response']
    item_recipe['ingredients'] = ingredients1
    item_recipe['ingredients_ner'] = ingredients1_arr
    for item2 in dataset_chicken_recipes:
        ingredients2 = str(recipe[recipe['id'] == item2['id']]['ingredients'].values[0])
        result2 = run_openai_task(labels, ingredients2)
        ingredients2_arr = result2['function_response'] 
        matched_words,matched_count = word_match_count(ingredients1_arr, ingredients2_arr)
        matched_percentage = (matched_count/len(ingredients1_arr))*100
        item_recipe['matched_recipes'].append({'id':item2['id'], 'recipe_name':item2['recipe_name'],'ingredients':ingredients2,'ingredients_ner':ingredients2_arr, 'matched_percentage':matched_percentage, 'matched_words':matched_words})        
    recipe_authenticity_result.append(item_recipe)

 2024-04-03 21:42:26,792 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
 2024-04-03 21:42:26,799 - INFO - function_to_call: <function recipe_ingredients at 0x11fbf0e00>
 2024-04-03 21:42:26,800 - INFO - function_args: {'product': ['chicken breasts', 'olive oil', 'lemon juice', 'garlic', 'dried oregano', 'black pepper', 'salad greens', 'cherry tomatoes', 'cucumber', 'red onion', 'feta cheese', 'Kalamata olives', 'red wine vinegar', 'extra-virgin olive oil', 'Dijon mustard']}
 2024-04-03 21:42:28,198 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
 2024-04-03 21:42:28,202 - INFO - function_to_call: <function recipe_ingredients at 0x11fbf0e00>
 2024-04-03 21:42:28,202 - INFO - function_args: {'product': ['chicken breasts', 'olive oil', 'pineapple chunks', 'broccoli florets', 'baby spinach leaves', 'red onions', 'balsamic vinegar', 'sugar', 'ground cinnamon']}
 2024-04-03 21:42:29,501 - INFO - HTTP Request: 

In [95]:
class MatchedRecipe(BaseModel):
    id: str
    recipe_name: str
    ingredients: str
    ingredients_ner: list = []
    matched_percentage: float
    matched_words: set = []

class RecipeAuthenticity(BaseModel):
    id: str
    recipe_name: str
    ingredients: str
    ingredients_ner: list = []
    matched_recipes:List[MatchedRecipe] = []

class RecipiesAuthenticity(BaseModel):
    recpies_authenticity: List[RecipeAuthenticity] = []

In [96]:
recpies = RecipiesAuthenticity()
for item in recipe_authenticity_result:
    recipe_authenticity = RecipeAuthenticity(id=item['id'], 
                                            recipe_name=item['recipe_name'],
                                            ingredients= item['ingredients'],
                                            ingredients_ner = item['ingredients_ner'])
    for item_matched in item['matched_recipes']:
        recipe_authenticity.matched_recipes.append(MatchedRecipe(id=str(item_matched['id']), 
                                            recipe_name=item_matched['recipe_name'],
                                            ingredients= item_matched['ingredients'],
                                            ingredients_ner = item_matched['ingredients_ner'],
                                            matched_percentage=item_matched['matched_percentage'],
                                            matched_words= item_matched['matched_words']))
    recpies.recpies_authenticity.append(recipe_authenticity)

In [97]:
import json
with open('../data/scrapped_recipes/chicken_salad.json', 'w') as f:
    json.dump(recpies.model_dump(mode='json'), f, indent=4, ensure_ascii=False)